# Check GPU options

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15929101096674432473
xla_global_id: -1
]


# Import needed libraries

In [12]:
# Import general purpose python libraries
import os
import matplotlib.pyplot as plt
from PIL import Image # For handling the images
import numpy as np
from sklearn import metrics
import pandas as pd
# Import different Keras functionalities
import tensorflow as tf
from tensorflow import keras
from keras.constraints import maxnorm
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD,Adagrad
# from keras.optimizers import SGD,Adagrad
from keras import backend as K
#from keras.applications import ResNet50,Xception,VGG16,VGG19
from sklearn.model_selection import train_test_split

####
# Where to save the figures
IMAGES_PATH = "../images"
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
from utils import saveFigures,write_json,create_json,load_previous_weights

from models.vgg16 import VGG16

# Visualize data

In [13]:
cifar10 = keras.datasets.cifar10
(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=1)

X_train = X_train/255.
X_valid = X_valid/255.
X_test = X_test/255.

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_test.shape[1]
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

print("num_training_samples: ", X_train.shape)

num_training_samples:  (40000, 32, 32, 3)


# Configuration parameters
Configuration values of different parts of the solution. You should change some of them to obtain better results.

In [16]:
from datetime import datetime
# Randomize the initial network 
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


# Directory where to store weights of the model and results
now = datetime.now()
current_time = now.strftime("%d_%H_%M_%S")
experiment_rootdir = "./test/"+current_time+"/"
weights_path = experiment_rootdir +"weights.h5"
json_path = experiment_rootdir +"experiment.json"

# Create experiment directory if it does not exists
if not os.path.exists(experiment_rootdir):
    os.makedirs(experiment_rootdir)
print(experiment_rootdir)

./test/04_18_22_39/


In [17]:
# Parameters that configures the training process

# Tunable parameters
name_model = "Prueba"
initial_lr = 1e-4 

l2_reg = 0.0
dropout = 0.0

compile_parameters = { "metrics": ['accuracy'], "optimizer": SGD(learning_rate=initial_lr)}

# TODO


## Constant parameters
batch_size = 512 
epochs = 100 
initial_epoch = 0 
num_classes = 10

'''
optimizer=  = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
'''

parameters = {
    "name_model":name_model,
    "compile_parameters" :str(compile_parameters) ,
    "batch_size": batch_size,
    "epochs": epochs, 
    "initial_epoch":initial_epoch, 
    "initial_lr": initial_lr,
    "num_classes": num_classes,
    "dropout": dropout

}
create_json({"parameters":parameters}, json_path)


In [18]:
def trainModel(model,compile_parameters,X_train, y_train, validation_data, epochs, batch_size,experiment_rootdir,weights_path,json_path):
    model.compile(loss='categorical_crossentropy', optimizer=compile_parameters['optimizer'], metrics=compile_parameters['metrics'])
    print(model.summary())
    history = model.fit(x= X_train, y= y_train, validation_data = validation_data, epochs = epochs, batch_size= batch_size)
    model.save_weights(weights_path)
    hist_df = pd.DataFrame(history.history) 
    data = {"train_result2":hist_df.to_dict()}
    write_json(data,json_path)
    return history

# Training 

In [19]:
#Get model
model = VGG16(dropout, num_classes, img_width=32, img_height=32, img_channels=3)
#load weights
load_previous_weights(model,weights_path)
#training model
val = (X_test, y_test)
history =trainModel(model,compile_parameters,X_train, y_train,val, epochs, batch_size,experiment_rootdir,weights_path,json_path)

Impossible to find weight path. Returning untrained model
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       147584    
   

2022-01-04 18:23:15.496110: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 491520000 exceeds 10% of free system memory.


Epoch 1/100
 1/79 [..............................] - ETA: 24:54 - loss: 2.3026 - accuracy: 0.0625

In [14]:
saveFigures(experiment_rootdir, history, "accuracy", "loss")

<Figure size 432x288 with 0 Axes>

In [17]:

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "batch_size": np.arange(10, 40, 10),
    "initial_lr": np.arange(0.00001, 0.0001, 0.00001),
    "dropout": np.arange(0.1, 0.9, 0.1)
}

rnd_search_cv = RandomizedSearchCV(model, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <keras.engine.training.Model object at 0x000002384B30F748> does not.

# Testing

In [15]:
model = VGG16(dropout, num_classes, img_width=32, img_height=32, img_channels=3)
# Load saved weights
model.compile(loss='categorical_crossentropy', optimizer=compile_parameters['optimizer'], metrics=compile_parameters['metrics'])

y_proba = model.predict(X_test)
y_pred = np.argmax(model.predict(X_test), axis=1)
''' 
# Create groundtruth
gt = np.zeros(y_pred.shape)
init_index = 0
end_index = 0
user_id = 0
for users in sorted(os.listdir(test_dir)):
    user_id = user_id + 1
    if os.path.isdir(os.path.join(test_dir, users)):
        user_path = os.path.join(test_dir, users)
        for root, _, files in sorted(os.walk(user_path)):
            num_images_user = len(files)
            end_index = init_index + num_images_user
            gt[init_index:end_index] = user_id - 1
            init_index = end_index
            
# Evaluate predictions: Average accuracy and highest errors
print("-----------------------------------------------")
print("Evaluation:")
# Compute average accuracy
ave_accuracy = metrics.accuracy_score(gt, y_pred)
print('Average accuracy = ', ave_accuracy)
print("-----------------------------------------------")

# Visualize confusion matrix                                           
plotcm.plotcm(experiment_rootdir, gt, y_pred,class_names, experiment_rootdir, normalize=True)
'''

' \n# Create groundtruth\ngt = np.zeros(y_pred.shape)\ninit_index = 0\nend_index = 0\nuser_id = 0\nfor users in sorted(os.listdir(test_dir)):\n    user_id = user_id + 1\n    if os.path.isdir(os.path.join(test_dir, users)):\n        user_path = os.path.join(test_dir, users)\n        for root, _, files in sorted(os.walk(user_path)):\n            num_images_user = len(files)\n            end_index = init_index + num_images_user\n            gt[init_index:end_index] = user_id - 1\n            init_index = end_index\n            \n# Evaluate predictions: Average accuracy and highest errors\nprint("-----------------------------------------------")\nprint("Evaluation:")\n# Compute average accuracy\nave_accuracy = metrics.accuracy_score(gt, y_pred)\nprint(\'Average accuracy = \', ave_accuracy)\nprint("-----------------------------------------------")\n\n# Visualize confusion matrix                                           \nplotcm.plotcm(experiment_rootdir, gt, y_pred,class_names, experiment_